In [8]:
from google.cloud import bigquery
from google.cloud.bigquery.client import Client
import os

## Access to BigQuery, Import data

In [9]:
# Create a "Client" object
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/iriswang/Google Drive/Stevens/summer2019/BigQuery/My Project 80150-b74d1b50b533.json'
client = Client()

### Exercise 1 - Hacker News dataset 

In [10]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [11]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there are four!)
for table in tables:  
    print(table.table_id)

comments
full
full_201510
stories


In [12]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

In [13]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema

[SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", ()),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', ()),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', ()),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', ()),
 SchemaField('title', 'STRING', 'NULLABLE', 'Story title', ()),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', ()),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', ()),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', ()),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', ()),
 SchemaField('deleted', 'BOOLEAN', 'NULLABLE', 'Is deleted?', ()),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', ()),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', ()),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", ()),
 SchemaField('ran

In [14]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,by,score,time,timestamp,title,type,url,text,parent,deleted,dead,descendants,id,ranking
0,Lordarminius,NaN,1503404140,2017-08-22 12:15:40+00:00,,comment,,70 Million Jobs featured Show HN recently. I r...,15071091.0,None,None,None,15071589,None
1,viraptor,NaN,1324642733,2011-12-23 12:18:53+00:00,,comment,,I guess there's some amount of domains where i...,3384132.0,None,None,None,3385476,None
2,cybernewsalerts,1.0,1396594940,2014-04-04 07:02:20+00:00,U.S. warns China not to attempt Crimea-style a...,story,http://www.cybernewsalerts.com/2014/04/us-warn...,,NaN,None,True,None,7529775,None
3,faragon,NaN,1500940066,2017-07-24 23:47:46+00:00,,comment,,"Please, Microsoft, don&#x27;t remove the Paint...",14837260.0,None,None,None,14843788,None
4,TeMPOraL,NaN,1488467798,2017-03-02 15:16:38+00:00,,comment,,&gt; <i>Modafinil is heavily stealth “marketed...,13774202.0,None,None,None,13774559,None


In [15]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

,by
0,Lordarminius
1,viraptor
2,cybernewsalerts
3,faragon
4,TeMPOraL


### Exercise 2 - Crime data in the city of Chicago.

In [17]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_crime" dataset
dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [23]:
### 1) Count tables in the dataset
#How many tables are in the Chicago Crime dataset?

tables = list(client.list_tables(dataset))

print(len(tables))

for table in tables:  
    print(table.table_id)

1
crime


In [25]:
num_tables = 1  # Store the answer as num_tables and then run this cell

In [26]:
### 2) Explore the table schema
#How many columns in the `crime` table have `TIMESTAMP` data?

# Construct a reference to the "full" table
table_ref = dataset_ref.table("crime")

# API request - fetch the table
table = client.get_table(table_ref)

In [27]:
table.schema

[SchemaField('unique_key', 'INTEGER', 'REQUIRED', 'Unique identifier for the record.', ()),
 SchemaField('case_number', 'STRING', 'NULLABLE', 'The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.', ()),
 SchemaField('date', 'TIMESTAMP', 'NULLABLE', 'Date when the incident occurred. this is sometimes a best estimate.', ()),
 SchemaField('block', 'STRING', 'NULLABLE', 'The partially redacted address where the incident occurred, placing it on the same block as the actual address.', ()),
 SchemaField('iucr', 'STRING', 'NULLABLE', 'The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.', ()),
 SchemaField('primary_type', 'STRING', 'NULLABLE', 'The primary description of the IUCR code.', ()),
 SchemaField('description', 'STRING', 'NULLABLE', 'The secondary description of the IUCR code, a subcategory of the primary descriptio

In [28]:
num_timestamp_fields = 2

In [42]:
### 3) Create a crime map
#create a map with a dot at the location of each crime, 
#what are the names of the two fields you likely need to pull out of the `crime` table to plot the crimes on a map?
client.list_rows(table, max_results=10).to_dataframe()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,11711587,JC292718,2019-06-04 20:00:00+00:00,072XX W THORNDALE AVE,1360,CRIMINAL TRESPASS,TO VEHICLE,STREET,False,False,...,41,10,26,NaN,NaN,2019,2019-06-11 16:16:50+00:00,NaN,NaN,None
1,11699492,JC278082,2019-05-25 13:57:00+00:00,036XX S ARCHER AVE,0557,ASSAULT,AGG PRO.EMP:KNIFE/CUTTING INST,PARK PROPERTY,True,False,...,12,59,04A,NaN,NaN,2019,2019-06-01 16:04:59+00:00,NaN,NaN,None
2,11684296,JC259726,2019-05-12 02:59:00+00:00,102XX S AVENUE M,1365,CRIMINAL TRESPASS,TO RESIDENCE,APARTMENT,False,False,...,10,52,26,1201495.0,1837768.0,2019,2019-05-19 16:04:45+00:00,41.709585,-87.537699,"(41.709585083, -87.537698669)"
3,11654387,JC223563,2019-04-13 11:30:00+00:00,061XX N OLCOTT AVE,4255,KIDNAPPING,UNLAWFUL INTERFERE/VISITATION,RESIDENCE,False,True,...,41,10,26,1125515.0,1940423.0,2019,2019-04-20 16:05:46+00:00,41.992877,-87.813678,"(41.992877184, -87.813677634)"
4,11652470,JC221158,2019-04-12 01:25:00+00:00,013XX E 87TH ST,5000,OTHER OFFENSE,OTHER CRIME AGAINST PERSON,STREET,True,False,...,8,45,26,1186543.0,1847562.0,2019,2019-04-19 16:10:08+00:00,41.736827,-87.592145,"(41.736826976, -87.592145254)"
5,11642582,JC209337,2019-03-03 09:00:00+00:00,068XX W DIVERSEY AVE,1110,DECEPTIVE PRACTICE,BOGUS CHECK,APARTMENT,False,False,...,36,18,11,1130162.0,1917853.0,2019,2019-04-05 15:59:08+00:00,41.930864,-87.797103,"(41.930864057, -87.797103496)"
6,11582512,JC136301,2019-02-01 11:35:00+00:00,131XX S LANGLEY AVE,3731,INTERFERENCE WITH PUBLIC OFFICER,OBSTRUCTING IDENTIFICATION,STREET,True,False,...,9,54,24,1183244.0,1818421.0,2019,2019-02-08 16:14:17+00:00,41.656938,-87.605134,"(41.656937576, -87.605134262)"
7,11595804,JC152512,2019-01-27 17:14:00+00:00,061XX W 64TH ST,0583,STALKING,CYBERSTALKING,OTHER,False,False,...,13,64,08A,1136646.0,1861456.0,2019,2019-02-17 15:58:51+00:00,41.775989,-87.774624,"(41.775988614, -87.774624211)"
8,11576016,JC127986,2019-01-24 09:45:00+00:00,048XX S KEELER AVE,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,RESIDENCE,False,False,...,14,57,11,1149177.0,1872236.0,2019,2019-01-31 16:11:34+00:00,41.805338,-87.728408,"(41.805338022, -87.728407787)"
9,11564602,JC114092,2019-01-12 11:40:00+00:00,054XX S PRINCETON AVE,0326,ROBBERY,AGGRAVATED VEHICULAR HIJACKING,STREET,False,False,...,3,37,03,1175212.0,1868809.0,2019,2019-01-23 11:26:56+00:00,41.795391,-87.633024,"(41.795391477, -87.633024431)"


In [43]:
fields_for_plotting = ['latitude', 'longitude']

## Basic queries

- Select, From, Where
- Group by, Having, Count
- Order by
- As & With
- Joining data

In [75]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# List all the tables in the "openaq" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset (there's only one!)
for table in tables:  
    print(table.table_id)

global_air_quality


In [47]:
# Construct a reference to the "global_air_quality" table
table_ref = dataset_ref.table("global_air_quality")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "global_air_quality" table
client.list_rows(table, max_results=5).to_dataframe()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


In [80]:
# 1)
# Query to select all the items from the "city" column where the "country" column is 'US'
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

In [49]:
# Create a "Client" object
client = bigquery.Client()

In [50]:
# Set up the query
query_job = client.query(query)

In [51]:
# API request - run the query, and return a pandas DataFrame
us_cities = query_job.to_dataframe()

In [52]:
# What five cities have the most measurements?
us_cities.city.value_counts().head()

Phoenix-Mesa-Scottsdale                     87
Houston                                     82
Los Angeles-Long Beach-Santa Ana            63
New York-Northern New Jersey-Long Island    60
Riverside-San Bernardino-Ontario            59
Name: city, dtype: int64

In [55]:
# 2)
query = """
        SELECT city, country
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

In [56]:
query = """
        SELECT *
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """

In [57]:
# Note: Triple quotes tell Python that everything inside them is a single string, 
#even though we have line breaks in it. The line breaks aren't necessary, but they make it easier to read your query.

## Working with big data

In [58]:
# Query to get the score column from every row where the type column has value "job"
query = """
        SELECT score, title
        FROM `bigquery-public-data.hacker_news.full`
        WHERE type = "job" 
        """

# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

This query will process 394726615 bytes.


In [67]:
# Only run the query if it's less than 100 MB
ONE_HUNDRED_MB = 1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_HUNDRED_MB)

# Set up the query (will only run if it's less than 100 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
safe_query_job.to_dataframe()

,score,title
0,4.0,Socialcam - iOS/Android Engineers
1,40.0,Airbnb (YC W09) is looking for summer/fall int...
2,4.0,Airbnb is hiring a UI designer
3,4.0,Airbnb hiring engineering interns
4,40.0,Posterous is hiring a Senior Infrastructure En...
5,7.0,Frontend Engineer - Mixpanel Analytics [SF]
6,4.0,Justin.tv is hiring Rails Engineers [SF/SOMA]
7,4.0,Airbnb - Fraud Management Engineer
8,8.0,"Well-funded, profitable YC company hiring engi..."
9,4.0,WePay - Operations Associate


In [66]:
# Only run the query if it's less than 1 GB
ONE_GB = 1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

# Set up the query (will only run if it's less than 1 GB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
job_post_scores = safe_query_job.to_dataframe()

# Print average score for job posts
job_post_scores.score.mean()

1.9628760215614676

## Exercise - global_air_quality

In [82]:
#### 1) Which countries have reported pollution levels in units of "ppm"? 

first_query = """
              SELECT DISTINCT country
              FROM `bigquery-public-data.openaq.global_air_quality`
              WHERE unit = "ppm"
              """

client = bigquery.Client()
query_job = client.query(first_query)
countries = query_job.to_dataframe()
countries.head()

,country
0,US
1,AU
2,CL
3,MX
4,BA


In [83]:
#### 2) Select all columns of the rows where pollution levels were reported to be exactly 0?
zero_pollution_query = """
                       SELECT *
                       FROM `bigquery-public-data.openaq.global_air_quality`
                       WHERE value = 0
                       """
query_job = client.query(zero_pollution_query)
zero_pollution_results = query_job.to_dataframe()
zero_pollution_results.head()

,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,Victoria Memorial - WBSPCB,Kolkata,IN,pm25,0.0,2017-10-16 20:45:00+00:00,µg/m³,CPCB,22.572645,88.363890,0.25
1,"Rabindra Bharati University, Kolkata - WBSPCB",Kolkata,IN,so2,0.0,2017-10-28 14:30:00+00:00,µg/m³,CPCB,22.627874,88.380400,0.25
2,"Końskie, MOBILNA",Końskie,PL,pm10,0.0,2018-12-21 13:00:00+00:00,µg/m³,GIOS,51.189526,20.408892,NaN
3,"Końskie, MOBILNA",Końskie,PL,pm25,0.0,2018-12-21 13:00:00+00:00,µg/m³,GIOS,51.189526,20.408892,NaN
4,Płock-Gimnazjum,Płock,PL,co,0.0,2019-06-18 08:00:00+00:00,µg/m³,GIOS,52.556280,19.687672,NaN
